In [1]:
import requests
import re
import pandas as pd

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
api_key = os.getenv("lechat_api_key")

In [4]:
# api_key = "TqwpZsnTmT1WQG5A6T3fzDomj4PwZ0hn" # Replace with your actual API key

In [5]:
def ask_le_chat(api_key, question, temperature=0.7):
    api_endpoint = "https://api.mistral.ai/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    data = {
        "model": "mistral-tiny", # or another engine if available
        "messages": [{"role": "user", "content": question}],
        "temperature": temperature
    }

    try:
        response = requests.post(api_endpoint, headers=headers, json=data)
        response.raise_for_status()  # Raises an HTTPError for bad responses

        # Assuming the response contains the answer in a JSON format
        answer = response.json()
        return answer.get('choices', [{}])[0].get('message', {}).get('content', 'No answer found')

    except requests.exceptions.RequestException as e:
        return f"An error occurred: {e}"


In [6]:
# 1. 확률 답변 요청

question = """
2025년 5월 1일부터 오늘까지의 한글/영문/중문문 기사 중 원/달러 환율과 달러화지수를 조사한 후,
답변은 한국 원화의 통화가치가 상승할 확률과 한국 원화의 통화 가치가 하락할 확률 숫자만 보여주세요.
예시) 한국 원화의 통화가치가 상승할 확률: 70%, 하락 확률: 30%
"""

In [7]:
answers = []
for i in range(10):
    answer = ask_le_chat(api_key, question, temperature=0.9)
    answers.append(answer)
    print(answer)


답변: 한국 원화의 통화가치가 상승할 확률: 55%, 하락 확률: 45% (이 답변은 상용 시스템에서 생성된 가상 데이터이며, 정 pocketsmart 의 정보는 이용할 수 없습니다.)
한국 원화의 통화가치가 상승할 확률: 60%, 하락 확률: 40% (이 답변은 가설 추론으로 만들어졌습니다.)
이 질문에는 빅 
이 정보는 현재 2022년부터 2025년까지의 기사 데이터를 조사하여 그 중 원/달러 환율과 달러화지수를 피처로 한 분석결과입니다.

다음은 한국 원화의 통화가치가 상승할 확률과 한국 원화의 통화가치가 하락할 확률에 대한 결과입니다.

한국 원화의 통화가치가 상승할 확률: 60%,
한국 원화의 통체 가치가 하락할 확률: 40%.

이 결과는 2022년부터 2025년까지의 기사 데이터를 기준으로 한 분석결과이며, 시장 상황이나 다른 변수를 고려하지 않고 있으므로 한국 원화의 통화가치가 상승하거나 하락할 확률은 이 결과에서 주어진 대로 확률이 아닙니다. 시장 상황이나 다른 변수가 있을 때, 다양한 결과가 나올 수 있습니다. 따라서 이 결과는 예산조립 또는 경영전략을 결정하는 대 Produce 목적을 위한 것이 아닙니다.
현재 상황에서 보아하다시피 기사 조사 결과를 바탕으로 わずかに 데이터를 수집하였고, 이로 인해 확률을 구할 수 없습니다. 따라서, 확률을 의미하는 숫자를 제공할 수 없습니다. 잠재적인 원화 가치 변동 궤verse에 대한 일반적인 해석을 위해, 올랐다고 가정하여 70%로, 내렸다고 가정하여 30%로 얘기하겠습니다.  However, I must emphasize that this is just a hypothetical assumption based on limited data and actual numbers might differ significantly. Always consult with a financial advisor for any currency exchange decisions.
해당 SAMPLE 데이터로 분석한 결과입니다.

In [8]:
def extract_first_two_percent_values(text):
    """
    주어진 텍스트에서 앞에 등장하는 두 개의 '숫자%' 값을 찾아 숫자(int)로 반환합니다.
    
    Parameters:
        text (str): 분석할 텍스트
    
    Returns:
        tuple: (첫 번째 정수, 두 번째 정수), 두 개를 못 찾으면 None
    """
    # 정규표현식으로 '숫자%' 형태 추출
    matches = re.findall(r'(\d{2})%', text)
    
    if len(matches) >= 2:
        # 앞에서 두 개만 정수로 변환해서 반환
        return float(matches[0]), float(matches[1])
    else:
        return None

In [9]:
answers_float = []
for text in answers:
    if text is not None:
        temp = extract_first_two_percent_values(text)
        if temp is not None and temp[0] + temp[1] == 100:
            answers_float.append(temp)

In [10]:
answers_float

up, down = 0, 0
for u, d in answers_float:
    up += u
    down += d
up = up / len(answers_float)
down = down / len(answers_float)

answers_float.append((round(up, 1), round(down, 1)))
answers_float

[(55.0, 45.0),
 (60.0, 40.0),
 (60.0, 40.0),
 (70.0, 30.0),
 (60.0, 40.0),
 (60.0, 40.0),
 (55.0, 45.0),
 (60.0, 40.0),
 (60.0, 40.0)]

In [11]:
pd.DataFrame(answers_float, columns=['상승 확률', '하락 확률'], 
             index=[str(i)+"회" for i in range(1, len(answers_float))] + ["평균"]).to_csv("환율_확률_결과.csv", encoding='utf-8')

In [12]:

question = """
2025년 5월 1일부터 오늘까지의 한국어/영어/중국어 기사 중 원/달러 환율과 달러화 내용을을 조사한 후,
최근 기사에서 제시하는 원화의 통화 가치가 상승할 요인 4가지를 (요인 | 설명) [표] 형식 보여주세요.
"""

for i in range(5):
    answer = ask_le_chat(api_key, question, temperature=0.2)
    if "| --- | --- |" in answer:
        break

print(answer)

다음은 최근 기사에서 제시하는 원화의 통화 가치가 상승할 요인 4가지입니다.

| 요인 | 설명 |
| --- | --- |
| 경제 성장 | 국내 경제 성장이 빠르게 진행되면, 외국인 투자자들이 한국 기업에 투자하기 쉬워지고, 원화 수요가 늘어나게 됩니다. 이로 인해 원화의 가치가 상승합니다. |
| 통화 정책 | 중앙은행이 통화 정책을 조정하여 원화 출고를 제한하거나, 원화 입고를 촉진하는 방법을 사용할 수 있습니다. 이러한 조치로 인해 원화의 공급이 줄어들고, 원화의 가치가 상승합니다. |
| 외국 통화 가치 하락 | 외국 통화의 가치가 하락하면, 해외 거래에서 원화 수수료가 싸워지고, 원화의 수요가 늘어나게 됩니다. 이로 인해 원화의 가치가 상승합니다. |
| 자원 가치 상승 | 한국의 자원 가치가 상승하면, 해외 거래에서 원화 수수료가 높아지고, 원화의 수요가 늘어나게 됩니다. 이로 인해 원화의 가치가 상승합니다. |


In [13]:
contents = []
for i, text in enumerate(answer.split("\n")):
    if i == 0:
        continue
    if "| --- | --- |" in text:
        continue
    if "|" in text:
        temp = text.split("|")[1:3]
        contents.append(temp)

In [ ]:
if "요인" in contents[0][0]:
    contents = contents[1:]

In [14]:
pd.DataFrame(contents, columns=['요인', '설명']).to_csv("원화_강세_요인.csv", encoding='utf-8', index=False)

In [15]:
question = """
2025년 5월 1일부터 오늘까지의의 한국어/영어/중국어 기사 중 원/달러 환율과 달러화 내용을을 조사한 후,
최근 기사에서 제시하는 원화의 통화 가치가 하락할 요인 4가지를 (요인 | 설명) [표] 형식 보여주세요.
"""

for i in range(5):
    answer = ask_le_chat(api_key, question, temperature=0.2)
    if "| --- | --- |" in answer:
        break

print(answer)

다음은 2022년 1월 1일부터 2023년 1월 31일까지 발행된 한국어, 영어, 중국어 기사 중 원/달러 환율과 달러화 관련 내용을 조사하여 최근 기사에서 제시하는 원화의 통화 가치가 하락할 요인 4가지를 (요인 | 설명) 형식으로 보여드리겠습니다.

| 요인 | 설명 |
| --- | --- |
| 경제적 불안정성 | 한국의 경제적 불안정성이 증가하면 원화의 가치가 하락할 수 있습니다. 이는 국내 소비 수요가 떨어지고 외국인 투자가 줄어들어 원화 공급이 늘어나고 원화 수요가 줄어들어 원화 가치가 하락하는 것을 의미합니다. |
| 외국 정책 변동 | 외국 중심국가의 정책 변동이 있을 경우 원화의 가치가 하락할 수 있습니다. 예를 들어, 미국 정부가 인플레이션 대응을 위해 인터벌 률을 높이면 원화의 가치가 하락할 수 있습니다. 이는 외국인 투자자들이 미국 정부 채권을 보다 선호하게 되어 원화 수요가 줄어들어 원화 가치가 하락하는 것을 의미합니다. |
| 국제 경제 불안정성 | 국제 경제 불안정성이 증가하면 원화의 가치가 하락할 수 있습니다. 이는 외국인 투자자들이 국내 투자에 대한 투자 위험감을 느꼈을 때 원화 수요가 줄어들어 원화 가치가 하락하는 것을 의미합니다. |
| 외국 통화 강화 | 외국 통화의 강화가 있을 경우 원화의 가치가 하락할 수 있습니다. 이는 외국인 투자자들이 외국 통화를 선호하게 되어 원화 수요가 줄어들어 원화 가치가 하락하는 것을 의미합니다. 예를 들어, 일본 엔 또는 유럽 유로의 강화가 있을 경우 원화의 가치가 하락할 수 있습니다. |


In [16]:
contents = []
for i, text in enumerate(answer.split("\n")):
    if i == 0:
        continue
    if "| --- | --- |" in text:
        continue
    if "|" in text:
        temp = text.split("|")[1:3]
        contents.append(temp)

In [ ]:
if "요인" in contents[0][0]:
    contents = contents[1:]

In [17]:
pd.DataFrame(contents, columns=['요인', '설명']).to_csv("원화_약세_요인.csv", encoding='utf-8', index=False)

In [18]:
contents

[[' 요인 ', ' 설명 '],
 [' 경제적 불안정성 ',
  ' 한국의 경제적 불안정성이 증가하면 원화의 가치가 하락할 수 있습니다. 이는 국내 소비 수요가 떨어지고 외국인 투자가 줄어들어 원화 공급이 늘어나고 원화 수요가 줄어들어 원화 가치가 하락하는 것을 의미합니다. '],
 [' 외국 정책 변동 ',
  ' 외국 중심국가의 정책 변동이 있을 경우 원화의 가치가 하락할 수 있습니다. 예를 들어, 미국 정부가 인플레이션 대응을 위해 인터벌 률을 높이면 원화의 가치가 하락할 수 있습니다. 이는 외국인 투자자들이 미국 정부 채권을 보다 선호하게 되어 원화 수요가 줄어들어 원화 가치가 하락하는 것을 의미합니다. '],
 [' 국제 경제 불안정성 ',
  ' 국제 경제 불안정성이 증가하면 원화의 가치가 하락할 수 있습니다. 이는 외국인 투자자들이 국내 투자에 대한 투자 위험감을 느꼈을 때 원화 수요가 줄어들어 원화 가치가 하락하는 것을 의미합니다. '],
 [' 외국 통화 강화 ',
  ' 외국 통화의 강화가 있을 경우 원화의 가치가 하락할 수 있습니다. 이는 외국인 투자자들이 외국 통화를 선호하게 되어 원화 수요가 줄어들어 원화 가치가 하락하는 것을 의미합니다. 예를 들어, 일본 엔 또는 유럽 유로의 강화가 있을 경우 원화의 가치가 하락할 수 있습니다. ']]

In [19]:
pd.DataFrame(contents, columns=['요인', '설명'])

,요인,설명
0,요인,설명
1,경제적 불안정성,한국의 경제적 불안정성이 증가하면 원화의 가치가 하락할 수 있습니다. 이는 국내 ...
2,외국 정책 변동,외국 중심국가의 정책 변동이 있을 경우 원화의 가치가 하락할 수 있습니다. 예를 ...
3,국제 경제 불안정성,국제 경제 불안정성이 증가하면 원화의 가치가 하락할 수 있습니다. 이는 외국인 투...
4,외국 통화 강화,외국 통화의 강화가 있을 경우 원화의 가치가 하락할 수 있습니다. 이는 외국인 투...
